In [1]:
import tensorflow as tf
from tensorflow import keras

import os
import matplotlib.pyplot as plt

In [4]:
url = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'
path_to_zip = keras.utils.get_file(fname='cats_and_dogs.zip',
                                   origin=url,
                                   extract=True,
                                   cache_dir='/content')

68608000/68606236 [==============================] - 0s 0us/step


In [5]:
data_path = os.path.join(os.path.dirname(path_to_zip), 'cats_and_dogs_filtered')
!find $data_path -type d

/content/datasets/cats_and_dogs_filtered
/content/datasets/cats_and_dogs_filtered/validation
/content/datasets/cats_and_dogs_filtered/validation/dogs
/content/datasets/cats_and_dogs_filtered/validation/cats
/content/datasets/cats_and_dogs_filtered/train
/content/datasets/cats_and_dogs_filtered/train/dogs
/content/datasets/cats_and_dogs_filtered/train/cats


In [6]:
base_dir = data_path
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

In [7]:
print(train_dir)
print(validation_dir)

/content/datasets/cats_and_dogs_filtered/train
/content/datasets/cats_and_dogs_filtered/validation


In [8]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [9]:
# 알아서 3차원 텐서로 만들어줘서 reshape 할 필요 없음
# 모든 이미지를 1/255로 스케일 조정
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

In [10]:
train_generator = train_datagen.flow_from_directory(
    # 타깃 디렉토리
    directory=train_dir,
    # 모든 이미지를 150 x 150 크기로 바꿉니다
    target_size=(150, 150),
    batch_size=20,
    shuffle=True,
    # binary_crossentropy 손실을 사용하기 때문에 이진 레이블이 필요
    class_mode='binary'
)

validation_generator = validation_datagen.flow_from_directory(
    directory=validation_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary'
)

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


ImportError: ignored

In [13]:
def build_model():
    model = keras.Sequential()
    model.add(keras.layers.Conv2D(32, (3, 3), activation='relu',
                                  input_shape=(150, 150, 3)))
    model.add(keras.layers.MaxPooling2D((2, 2)))
    model.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(keras.layers.MaxPooling2D((2, 2)))
    model.add(keras.layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(keras.layers.MaxPooling2D((2, 2)))
    model.add(keras.layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(keras.layers.MaxPooling2D((2, 2)))

    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(512, activation='relu'))
    model.add(keras.layers.Dense(1, activation='sigmoid'))
    model.summary()
    model.compile(loss='binary_crossentropy',
                  optimizer='rmsprop',
                  metrics=['accuracy'])
    return model

In [16]:
model = build_model()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 15, 15, 128)      

893 = (3 * 3) * 3 * 32
2차원 필터 크기 * 컬러 필터 depth * 필터 개수

In [17]:
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator) - 1,
    epochs=20,
    validation_data=validation_generator,
    validation_steps=len(validation_generator)
)

Epoch 1/20
99/99 [==============================] - 10s 105ms/step - loss: 0.7378 - accuracy: 0.5025 - val_loss: 0.6865 - val_accuracy: 0.5730
Epoch 2/20
99/99 [==============================] - 10s 105ms/step - loss: 0.6920 - accuracy: 0.5682 - val_loss: 0.6718 - val_accuracy: 0.6050
Epoch 3/20
99/99 [==============================] - 10s 105ms/step - loss: 0.6533 - accuracy: 0.6460 - val_loss: 0.8073 - val_accuracy: 0.5560
Epoch 4/20
99/99 [==============================] - 10s 104ms/step - loss: 0.6208 - accuracy: 0.6702 - val_loss: 0.6469 - val_accuracy: 0.6510
Epoch 5/20
99/99 [==============================] - 10s 105ms/step - loss: 0.5777 - accuracy: 0.7278 - val_loss: 0.5443 - val_accuracy: 0.7280
Epoch 6/20
99/99 [==============================] - 10s 106ms/step - loss: 0.5217 - accuracy: 0.7465 - val_loss: 0.6628 - val_accuracy: 0.6630
Epoch 7/20
99/99 [==============================] - 10s 105ms/step - loss: 0.4870 - accuracy: 0.7758 - val_loss: 0.6785 - val_accuracy: 0.6770

In [18]:
test_loss, test_acc = model.evaluate(validation_generator)

50/50 [==============================] - 3s 62ms/step - loss: 1.5315 - accuracy: 0.7630


In [19]:
data_batch, labels_batch = next(train_generator)
print('배치 데이터 크기:', data_batch.shape)
print('배치 레이블 크기:', labels_batch.shape)

배치 데이터 크기: (20, 150, 150, 3)
배치 레이블 크기: (20,)


In [20]:
callbacks = [
    keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3),
    keras.callbacks.ModelCheckpoint(filepath='./checkpoint', monitor='val_accuracy',
                                    save_best_only=True)
]

In [21]:
callback_model = build_model()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 15, 15, 128)      

In [23]:
history = callback_model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=30,
    validation_data=validation_generator,
    validation_steps=len(validation_generator),
    callbacks=callbacks
)

Epoch 1/30
100/100 [==============================] - ETA: 0s - loss: 0.8305 - accuracy: 0.5190WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/tracking/tracking.py:111: Model.state_updates (from tensorflow.python.keras.engine.training) is deprecated and will be removed in a future version.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: ./checkpoint/assets
100/100 [==============================] - 12s 122ms/step - loss: 0.8305 - accuracy: 0.5190 - val_loss: 0.6900 - val_accuracy: 0.5000
Epoch 2/30
100/100 [==============================] - 12s 119ms/step - loss: 0.7049 - accuracy: 0.5520 - val_loss: 0.6518 - val_accuracy: 0.6170
Epoch 3/30
100/100 [==============================] - 11s 108ms/step - loss: 0.6688 - accuracy: 0.6

In [24]:
callback_test_loss, callback_test_acc = callback_model.evaluate(validation_generator)

50/50 [==============================] - 3s 64ms/step - loss: 0.7898 - accuracy: 0.7040


In [25]:
loaded_model = keras.models.load_model('/content/checkpoint')

In [26]:
test_loss, test_acc = loaded_model.evaluate(validation_generator)

50/50 [==============================] - 3s 65ms/step - loss: 0.6049 - accuracy: 0.7510
